# Experiments

In [1]:
import pandas as pd
import numpy as np
from modules.functions import run_linear_regression
from modules.functions import run_polynomial_regression
from modules.functions import run_random_forest
from modules.functions import to_hours
from modules.functions import scale_data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# import csv file with the proper dtypes
dtype_dict = {
 'branded_code_share': 'string',
 'mkt_carrier': 'string',
 'mkt_carrier_fl_num': 'string',
 'op_unique_carrier': 'string',
 'tail_num': 'string',
 'op_carrier_fl_num': 'string',
 'origin_airport_id': 'string',
 'origin': 'string',
 'origin_city_name': 'string',
 'dest_airport_id': 'string',
 'dest': 'string',
 'dest_city_name': 'string',
 'crs_dep_time': 'int64',
 'dep_time': 'int64',
 'dep_delay': 'int64',
 'taxi_out': 'int64',
 'wheels_off': 'int64',
 'wheels_on': 'int64',
 'taxi_in': 'int64',
 'crs_arr_time': 'int64',
 'arr_time': 'int64',
 'arr_delay': 'int64',
 'crs_elapsed_time': 'int64',
 'actual_elapsed_time': 'int64',
 'air_time': 'int64',
 'distance': 'int64'}
flights = pd.read_csv("../LHL_Midterm_Project_Predicting_Flight_Delays/data/ransmpl_clean.csv", parse_dates=[0], dtype=dtype_dict)
df_no_outliers = pd.read_csv("../LHL_Midterm_Project_Predicting_Flight_Delays/data/ransmpl_clean_no_outliers.csv", parse_dates=[0], dtype=dtype_dict)
flights_test = pd.read_csv("../LHL_Midterm_Project_Predicting_Flight_Delays/data/flights_test.csv", parse_dates=[0], dtype=dtype_dict)

In [4]:
flights.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,833,-2,13,846,838,2,851,840,-11,76,67,52,304
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1537,66,16,1553,1648,3,1559,1651,52,88,74,55,331
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,557,-6,18,615,746,19,809,805,-4,126,128,91,632
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1129,-6,11,1140,1312,12,1352,1324,-28,137,115,92,728
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,1812,-8,11,1823,1936,6,2002,1942,-20,162,150,133,842


In [5]:
pd.set_option('display.max_rows', 50)

In [6]:
flights.sort_values("tail_num").head(1000)

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
299644,2019-07-22,G4,G4,602,G4,215NV,602,14314,PVU,"Provo, UT",15376,TUS,"Tucson, AZ",1404,1402,-2,11,1413,1431,7,1443,1438,-5,99,96,78,560
149815,2019-01-14,G4,G4,1207,G4,215NV,1207,14082,PGD,"Punta Gorda, FL",10431,AVL,"Asheville, NC",958,948,-10,10,958,1123,7,1138,1130,-8,100,102,85,588
259818,2019-04-10,G4,G4,1774,G4,215NV,1774,11697,FLL,"Fort Lauderdale, FL",10135,ABE,"Allentown/Bethlehem/Easton, PA",650,647,-3,7,654,919,5,942,924,-18,172,157,145,1041
78684,2018-12-02,G4,G4,858,G4,215NV,858,14112,PIE,"St. Petersburg, FL",11193,CVG,"Cincinnati, OH",1410,1355,-15,13,1408,1556,4,1620,1600,-20,130,125,108,776
371787,2019-06-04,G4,G4,1821,G4,215NV,1821,12339,IND,"Indianapolis, IN",14112,PIE,"St. Petersburg, FL",1618,1612,-6,8,1620,1805,4,1829,1809,-20,131,117,105,840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25957,2019-08-22,G4,G4,2450,G4,230NV,2450,11986,GRR,"Grand Rapids, MI",14112,PIE,"St. Petersburg, FL",1607,1607,0,15,1622,1835,5,1842,1840,-2,155,153,133,1044
193469,2018-09-09,G4,G4,1776,G4,230NV,1776,11697,FLL,"Fort Lauderdale, FL",12544,USA,"Concord, NC",1211,1210,-1,18,1228,1403,9,1410,1412,2,119,122,95,643
193471,2018-09-09,G4,G4,1703,G4,230NV,1703,12945,LEX,"Lexington, KY",11697,FLL,"Fort Lauderdale, FL",903,903,0,6,909,1107,6,1121,1113,-8,138,130,118,865
193933,2018-09-10,G4,G4,1713,G4,230NV,1713,11996,GSP,"Greer, SC",11697,FLL,"Fort Lauderdale, FL",2119,2110,-9,12,2122,2251,7,2310,2258,-12,111,108,89,620


In [7]:
feats = flights[["fl_date", "mkt_carrier", "mkt_carrier_fl_num", "origin", "dest"]].copy()

In [8]:
feats["count"] = 1

feats["op_unique_carrier"] = flights["op_unique_carrier"].copy()
feats["year"] = feats["fl_date"].dt.year.astype("object")
feats["month"] = feats["fl_date"].dt.month.astype("object")
feats["day"] = feats["fl_date"].dt.day.astype("object")
feats["day_of_week"] = feats["fl_date"].dt.weekday.astype("object")

feats["air_time"] = flights["air_time"].copy()
feats["distance"] = flights["distance"].copy()

feats["crs_dep_hour"] = to_hours(flights["crs_dep_time"]).round().replace(24, 0)
feats["crs_arr_hour"] = to_hours(flights["crs_arr_time"]).round().replace(24, 0)

/Users/zacharyargentin/opt/anaconda3/envs/bootcamp/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


**avg delay by carrier feature**

In [9]:
# calculate avg delay by carrier
avg_delay_df = flights[["op_unique_carrier", "arr_delay"]].groupby("op_unique_carrier").mean().reset_index()
# round values and create a dictionary mapping each carrier to their avg delay
avg_delay_map = dict(avg_delay_df.values)
rounded_delay_vals = [round(val, 2) for val in avg_delay_map.values()]
avg_delay_map = dict(zip(avg_delay_map.keys(), rounded_delay_vals))
# creating new column based on the mapping
feats["avg_delay_for_carrier"] = flights["op_unique_carrier"].map(avg_delay_map)

**num of flights that take off at given time of day feature**

In [10]:
# creating a 'traffic by hour' feature (dep flights)
n_flights_dep = feats[["crs_dep_hour", "count"]].groupby("crs_dep_hour").sum()
n_flights_dep.reset_index(inplace=True)
n_flights_dep_map = dict(zip(n_flights_dep["crs_dep_hour"], n_flights_dep["count"]))
feats["dep_traffic"] = feats["crs_dep_hour"].map(n_flights_dep_map)

**number of flights that land at given time of day feature**

In [11]:
# creating a 'traffic by hour' feature (arr_flights)
n_flights_arr = feats[["crs_arr_hour", "count"]].groupby("crs_arr_hour").sum()
n_flights_arr.reset_index(inplace=True)
n_flights_arr_map = dict(zip(n_flights_arr["crs_arr_hour"], n_flights_arr["count"]))
feats["arr_traffic"] = feats["crs_arr_hour"].map(n_flights_arr_map)

In [12]:
###
avg_delay_by_city = flights[["dest_city_name", "arr_delay"]].groupby("dest_city_name").mean()
avg_delay_by_city

,arr_delay
dest_city_name,
"Aberdeen, SD",19.111111
"Abilene, TX",6.156250
"Adak Island, AK",-6.200000
"Aguadilla, PR",10.313725
"Akron, OH",11.596059
...,...
"Worcester, MA",3.393939
"Wrangell, AK",-4.515152
"Yakima, WA",1.086957


In [13]:
feats.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,count,op_unique_carrier,year,month,day,day_of_week,air_time,distance,crs_dep_hour,crs_arr_hour,avg_delay_for_carrier,dep_traffic,arr_traffic
0,2018-03-07,AA,465,PHX,SAN,1,AA,2018,3,7,2,52,304,9.0,9.0,6.07,22293,23025
1,2018-03-07,AA,591,CLT,DCA,1,AA,2018,3,7,2,55,331,15.0,16.0,6.07,23703,23854
2,2018-03-07,AA,600,FLL,CLT,1,AA,2018,3,7,2,91,632,6.0,8.0,6.07,26483,16692
3,2018-03-07,AA,1805,CLT,BOS,1,AA,2018,3,7,2,92,728,12.0,14.0,6.07,24118,23869
4,2018-03-07,AA,2615,CLT,TUL,1,AA,2018,3,7,2,133,842,18.0,20.0,6.07,24517,21411


In [14]:
df2 = flights[['fl_date', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance']].copy()
df2.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,851,76,304
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1559,88,331
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,809,126,632
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1352,137,728
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,2002,162,842


# Feature Engineering

In [15]:
df2["year"] = df2["fl_date"].dt.year.astype("object")
df2["month"] = df2["fl_date"].dt.month.astype("object")
df2["day"] = df2["fl_date"].dt.day.astype("object")
df2["day_of_week"] = df2["fl_date"].dt.weekday.astype("object")

In [16]:
df2["fl_date"].dt.weekday

0         2
1         2
2         2
3         2
4         2
         ..
390053    2
390054    2
390055    2
390056    2
390057    2
Name: fl_date, Length: 390058, dtype: int64

In [17]:
df2.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,851,76,304,2018,3,7,2
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1559,88,331,2018,3,7,2
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,809,126,632,2018,3,7,2
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1352,137,728,2018,3,7,2
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,2002,162,842,2018,3,7,2


In [18]:
df_num = df2[["month", "day_of_week"]]
df_num = pd.get_dummies(df_num)
df_num

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390053,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390054,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390055,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390056,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [19]:
feats.head(2)

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,count,op_unique_carrier,year,month,day,day_of_week,air_time,distance,crs_dep_hour,crs_arr_hour,avg_delay_for_carrier,dep_traffic,arr_traffic
0,2018-03-07,AA,465,PHX,SAN,1,AA,2018,3,7,2,52,304,9.0,9.0,6.07,22293,23025
1,2018-03-07,AA,591,CLT,DCA,1,AA,2018,3,7,2,55,331,15.0,16.0,6.07,23703,23854


# Select Features

In [20]:
y = flights["arr_delay"]
y2 = df_no_outliers["arr_delay"]

X1 = flights[["air_time", "distance"]]
X2 = df_num
X3 = df_no_outliers[["air_time", "distance"]]
X4 = feats[["crs_dep_hour", "crs_arr_hour", "avg_delay_for_carrier", "dep_traffic", "arr_traffic"]]
X5 = feats[["crs_dep_hour", "crs_arr_hour", "avg_delay_for_carrier", "dep_traffic", "arr_traffic",
            "air_time", "distance"]] # Current Best

# Test Models

In [21]:
# Linear Regression
lr, y_pred, r2_train, r2_test = run_linear_regression(X5, y)
print(r2_train, r2_test)

0.024628583482650512 0.0232277572322247


In [22]:
# Polynomial Regression
for degree in range(2,6):
    pr, y_pred, r2_train, r2_test = run_polynomial_regression(X5, y, degree)

---Results---
degree = 2
Train score = 0.030960215354540854
Test score = 0.028276882254258395
---Results---
degree = 3
Train score = 0.03319622043809012
Test score = 0.029775665473524837
---Results---
degree = 4
Train score = 0.035276933668353716
Test score = 0.0253509658472284
---Results---
degree = 5
Train score = 0.0403292843912576
Test score = -0.15871967585701507


In [23]:
# Random Forest
rf, y_pred, r2_train, r2_test = run_random_forest(X4, y, n_estimators=200, max_depth=5)
print(r2_train, r2_test)

train score = 0.018348921031598908
test score = 0.01549814707943431
0.018348921031598908 0.01549814707943431


In [24]:
flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [25]:
flights_test["count"] = 1

flights_test["year"] = flights_test["fl_date"].dt.year.astype("object")
flights_test["month"] = flights_test["fl_date"].dt.month.astype("object")
flights_test["day"] = flights_test["fl_date"].dt.day.astype("object")
flights_test["day_of_week"] = flights_test["fl_date"].dt.weekday.astype("object")

flights_test["crs_dep_hour"] = to_hours(flights_test["crs_dep_time"]).round().replace(24, 0)
flights_test["crs_arr_hour"] = to_hours(flights_test["crs_arr_time"]).round().replace(24, 0)

/Users/zacharyargentin/opt/anaconda3/envs/bootcamp/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
# calculate avg delay by carrier
avg_delay_df = flights[["op_unique_carrier", "arr_delay"]].groupby("op_unique_carrier").mean().reset_index()
# round values and create a dictionary mapping each carrier to their avg delay
avg_delay_map = dict(avg_delay_df.values)
rounded_delay_vals = [round(val, 2) for val in avg_delay_map.values()]
avg_delay_map = dict(zip(avg_delay_map.keys(), rounded_delay_vals))
# creating new column based on the mapping
flights_test["avg_delay_for_carrier"] = flights_test["op_unique_carrier"].map(avg_delay_map)

In [29]:
# creating a 'traffic by hour' feature (dep flights)
n_flights_dep = feats[["crs_dep_hour", "count"]].groupby("crs_dep_hour").sum()
n_flights_dep.reset_index(inplace=True)
n_flights_dep_map = dict(zip(n_flights_dep["crs_dep_hour"], n_flights_dep["count"]))
flights_test["dep_traffic"] = flights_test["crs_dep_hour"].map(n_flights_dep_map)

In [30]:
# creating a 'traffic by hour' feature (arr_flights)
n_flights_arr = feats[["crs_arr_hour", "count"]].groupby("crs_arr_hour").sum()
n_flights_arr.reset_index(inplace=True)
n_flights_arr_map = dict(zip(n_flights_arr["crs_arr_hour"], n_flights_arr["count"]))
flights_test["arr_traffic"] = flights_test["crs_arr_hour"].map(n_flights_arr_map)

In [31]:
flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,count,year,month,day,day_of_week,crs_dep_hour,crs_arr_hour,avg_delay_for_carrier,dep_traffic,arr_traffic
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,1,2020,1,1,2,18.0,20.0,3.53,24517,21411
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363,1,2020,1,1,2,11.0,13.0,3.53,23686,22794
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333,1,2020,1,1,2,20.0,21.0,3.53,20979,23548
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333,1,2020,1,1,2,13.0,15.0,3.53,22409,22471
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333,1,2020,1,1,2,9.0,11.0,3.53,22293,24820


In [32]:
X = flights_test[["crs_dep_hour", "crs_arr_hour", "avg_delay_for_carrier", "dep_traffic",
                  "arr_traffic", "distance"]]

In [34]:
# Linear Regression
predictions = lr.predict(X)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 7 is different from 6)